In [1]:
input_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\TextualPage4.pdf" # Replace with your input PDF file path
output_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\sample.pdf"  # Replace with your desired output PDF file path
file_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\files\SamcoFactSheet2024.pdf"

In [4]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\TextualPdf.pdf"

#text

In [5]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\TabularPdf.pdf"


In [3]:
import fitz 
import camelot
import statistics

In [3]:
import fitz  # PyMuPDF

def extract_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Usage
pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\TextualPage4.pdf"
text_data = extract_text(pdf_path)


In [ ]:


def detect_columns(pdf_path, eps=50, min_samples=2):
    """
    Detect the number of text columns in each page of a PDF.

    Parameters:
    - pdf_path: str, path to the PDF file.
    - eps: float, maximum distance between two samples for them to be considered in the same neighborhood (DBSCAN parameter).
    - min_samples: int, number of samples in a neighborhood for a point to be considered a core point (DBSCAN parameter).

    Returns:
    - List of integers representing the number of columns detected on each page.
    """
    doc = fitz.open(pdf_path)
    column_counts = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text_dict = page.get_text("dict")

        # Collect x-coordinates of text blocks
        x_coords = []
        for block in text_dict.get("blocks", []):
            bbox = block.get("bbox", [])
            if bbox:
                x0 = bbox[0]  # x-coordinate of the left side of the block
                x_coords.append([x0])

        if x_coords:
            # Convert to numpy array
            x_coords = np.array(x_coords)

            # Apply DBSCAN clustering
            clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(x_coords)

            # Number of clusters (excluding noise if present)
            num_columns = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
            column_counts.append(num_columns)
        else:
            column_counts.append(0)

    doc.close()
    return column_counts

# Example usage
pdf_path = file_path  # Replace with your PDF file path
columns_per_page = detect_columns(pdf_path)
for i, num_columns in enumerate(columns_per_page):
    print(f"Page {i + 1}: {num_columns} columns detected")


In [62]:

def annotate_pdf(input_path, output_path):
    # Open the PDF document
    pdf = fitz.open(input_path)
    
    # Iterate through each page
    for page_num in range(len(pdf)):
        page = pdf.load_page(page_num)
        # Extract text as a dictionary containing blocks, lines, and spans
        text_dict = page.get_text("dict")
        
        # Iterate through blocks
        for block in text_dict.get("blocks", []):
            # Check if the block contains lines
            if "lines" in block:
                # Iterate through lines
                for line in block["lines"]:
                    # Get the bounding box of the line
                    line_bbox = fitz.Rect(line["bbox"])
                    # Draw a rectangle around the line
                    page.draw_rect(line_bbox, color=(1, 0, 0), width=0.5)
    
    # Save the annotated PDF to the output path
    pdf.save(output_path)
    pdf.close()

# Example usage
input_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\TextualPage4.pdf" # Replace with your input PDF file path
output_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\sample.pdf"  # Replace with your desired output PDF file path
annotate_pdf(input_pdf_path, output_pdf_path)


In [6]:
def extract_pdf(path):
    
    pdf = fitz.open(path)
    page_content = []
    
    for pgn in range(len(pdf)):
        page = pdf.load_page(pgn)
        text = page.get_text("dict")
        page_content.append(text)
    return page_content
        
    # sort_by_y = []
    # for block in page_content:
    #     for lines in block['lines']:
    #         for span in lines['span']
    
    
content = extract_pdf(input_pdf_path)
print(f"\nPages:{len(content)}")


Pages:1


In [ ]:
firstPage = content[0]
#print all the content of the block
print(f"\nLength of Blocks: {len(firstPage['blocks'])}")
for block in firstPage['blocks']:
    for spans in block['lines']:
        txt = " ".join(item['text'] for item in spans['spans'])
        print(block['bbox'],txt, spans)

In [ ]:
#sort Left to Right Top to Bottom
# x0, y0 lower-left x1, y1 upper-right

SortedBlocks = sorted(firstPage['blocks'], key= lambda x : (x['bbox'][1], x['bbox'][0]))

block_sizes = {}
block_fonts = {}
for block in SortedBlocks:
    for spans in block['lines']:
        temp = []
        txt = " ".join(item['text'] for item in spans['spans'])
        for span in spans['spans']:
            temp.append(span['size'])
            #get size 
            size  = span['size']
            if size not in block_sizes.keys():
                block_sizes[size] = 1
            else:
                block_sizes[size] +=1
            
            #get font agg
            color = span['color']
            if color not in block_fonts.keys():
                block_fonts[color] = 1
            else:
                block_fonts[color]+=1
        print(block['bbox'],txt, "size:",temp)
        
        
sorted_blk_fonts = sorted(block_fonts.items(), key=lambda item: item[1])
sorted_blk_sizes = sorted(block_sizes.items(), key=lambda item: item[1])
print("\nSize: ", sorted_blk_sizes)
print("\nColors: ", sorted_blk_fonts)

In [63]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sizes = [i[0] for i in sorted_blk_sizes]
colors = [i[0] for i in sorted_blk_fonts]

X,Y = len(colors), len(sizes)

matrix_keys  = [(i,j) for i in colors for j in sizes]
matrix = {}

for block in SortedBlocks:
    # count occrences of (color, size) on each page block by block
    for spans in block['lines']:
        for span in spans['spans']:
            size = span['size']
            color = span['color']
            matrix[(color,size)] = matrix.get((color,size), 0) + 1


data = sorted(matrix.items(), key  = lambda x: x[1], reverse=True)

In [75]:
data[0]

((-12368828, 7.0), 52)

In [ ]:
#print the heatmap
df = pd.DataFrame(
    {
        'x': [pt[0][0] for pt in data],
        'y': [pt[0][1] for pt in data],
        'value': [pt[1]     for pt in data],
    }
)

# 2. Pivot so that rows = y, columns = x, and cell entries = 'value'
heatmap_df = df.pivot(index='y', columns='x', values='value')

plt.imshow(heatmap_df, origin="upper", cmap="inferno")

plt.colorbar(label='Value')
plt.title("Heatmap via Pandas Pivot")
plt.xlabel("X coordinate")
plt.ylabel("Y coordinate")

plt.xticks(ticks=range(len(heatmap_df.columns)), labels=heatmap_df.columns, rotation=45, ha="right")
plt.yticks(ticks=range(len(heatmap_df.index)),   labels=heatmap_df.index)

plt.tight_layout()
plt.show()

In [76]:
"""
string title : the principle title of the page
list headers: (txt, bbox, size, color) of all headers with side <title
list paras: (txt, bbox, size, color) of all paras with highest size occurence
list other: (txt, bbox, size, color)  of all the other chars, txt

"""

#Solely based on color
sorted_block_by_color = {key[0]: [] for key in sorted_blk_fonts}
sorted_blocks_by_size = {key[0]: [] for key in sorted_blk_sizes}
sorted_by_both = {(key[0][0],key[0][1]): [] for key in data} #sorted wrt color and size


for blocks in SortedBlocks:
    text = []
    for spans in blocks['lines']:
        txt = " ".join(item['text'] for item in spans['spans'])
        for span in spans['spans']:
            color = span['color']
            sorted_block_by_color[color].append(txt)
            size = span['size']
            sorted_blocks_by_size[size].append(txt)
            
            sorted_by_both[(color,size)].append(txt)

In [ ]:
sorted_block_by_color

In [ ]:
sorted_blocks_by_size

In [90]:
for item in sorted_by_both:
    print(f"\n{item[0]}")
    for text in item[1]:
        print(f"\n{text}")


(-12684359, 24.0)

Samco Active Momentum Fund

(-12303035, 8.0)

(An open-ended equity scheme following momentum theme)

(-12220216, 6.0)

exposure of

(-1, 6.0)

NAV as on 31  st October 2024 (· per unit)

(-1, 10.0)

3

(-12220216, 8.0)

Mr. Paras Matalia,

Mr. Umeshkumar Mehta,

Mr. Dhawal Ghanshyam Dhanani

(-12368828, 6.0)

(Date of Allotment)

(With effect from October 03, 2024)

as on October 31,

2024

Lower of sales or purchases divided by average AUM for last rolling 12 months

(-1, 9.0)

Investment Objective

Portfolio as on October 31, 2024

Scheme Details

Fund Manager

NAV as on 31  st October 2024 (· per unit)

NAV as on 31  st October 2024 (· per unit)

Quantitative Data

Assets Under Management (AUM)

(-12303035, 7.0)

The investment objective of the Scheme is to seek to

generate long-term capital appreciation by investing in

stocks showing strong momentum. Momentum stocks are

such that exhibit positive price momentum · based on the

phenomenon that stocks which ha